<a href="https://www.kaggle.com/code/yujansaya/chemensemble-molecular-binding-with-ensemble?scriptVersionId=173383244" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1>Putting on my chemist robe and glasses :)</h1><center><img src="https://static.displate.com/280x392/displate/2022-12-28/494bd2c86cb966c2129375457a82982f_80fd640e09154845b9ca853fda67a96d.jpg" ></center>Let us first figure out what they want from us and what we want from our code. <br><br>


The dataset comprises binary classification data representing whether small molecules bind to three different protein targets. Each entry includes SMILES representations of molecule structures and binary labels for binding to each protein target. The competition data, provided by Leash Biosciences, consists of approximately **98M** training examples per protein, **200K** validation examples per protein, and **360K** test molecules per protein. *Keep in mind that dataset is very imbalanced:  roughly 0.5% of examples are classified as binders*<br><br>

<center><h2>So, what is SMILES?</h2><img src="https://static.wixstatic.com/media/7cced3_08c22a1ad56a4d2b81c46c8d71ebc34b~mv2.gif" ></center><br>

SMILES (Simplified Molecular Input Line Entry System) is a notation system for representing chemical structures in a computer-readable format. Developed with funding from the U.S. Environmental Protection Agency, it offers a flexible and easily learned approach to representing molecules. SMILES notation follows five basic syntax rules:

* Atoms and Bonds: Atoms are represented by their atomic symbols, with lowercase letters indicating aromatic atoms. Bonds are denoted by symbols (- for single, = for double, # for triple, * for aromatic, and . for disconnected structures).
* Simple Chains: Chains of atoms are represented by combining atomic symbols and bond symbols. Hydrogen atoms are suppressed unless explicitly stated.
* Branches: Branches from chains are enclosed in parentheses and placed directly after the atom to which they are connected.
* Rings: Ring structures are identified by using numbers to denote the opening and closing ring atoms. Different numbers are used for each ring, and bond symbols may precede the ring closure number.
* Charged Atoms: Charges on atoms are indicated by placing the atom symbol within brackets, enclosing the charge.<br>


<center><h2>What are the targets?</h2></center><br>

The dataset includes three protein targets:
<h3>EPHX2 (sEH):</h3><center><img src="https://www.researchgate.net/publication/26836510/figure/fig1/AS:394310370512898@1471022326590/Pathways-of-EETs-synthesis-metabolism-and-action.png" ></center><br>
* This target refers to epoxide hydrolase 2, encoded by the EPHX2 genetic locus. Its protein product, commonly known as soluble epoxide hydrolase (sEH), is an enzyme that catalyzes certain chemical reactions and hydrolyzes phosphate groups. It is a potential drug target for conditions like high blood pressure and diabetes. The dataset includes screening data obtained from Leash Biosciences, along with structural information for model evaluation.
<h3>BRD4:</h3><center><img src="https://ars.els-cdn.com/content/image/1-s2.0-S1043661823001238-ga1.jpg" ></center><br>
* Bromodomain 4 is encoded by the BRD4 locus. Its protein product, also named BRD4, plays a role in gene transcription regulation by binding to histones in the nucleus. This protein is implicated in cancer progression, and inhibiting its activity has been explored as a therapeutic strategy. The dataset includes screening data from Leash Biosciences and structural information for model evaluation.
<h3>ALB (HSA):</h3><center><img src="https://ars.els-cdn.com/content/image/1-s2.0-S0162013419301734-gr4.jpg" ></center><br>
* Serum albumin, encoded by the ALB locus, is the most abundant protein in blood. It regulates osmotic pressure and transports various molecules, including drugs, hormones, and fatty acids. Predicting the binding of small molecules to albumin is crucial for drug development, as it impacts drug distribution and effectiveness. The dataset includes screening data from Leash Biosciences and structural information for model evaluation.


<h3>Now let's get the things done! :D</h3> <br>

We shall start with the installation of necessary tools. <br><br>
**DuckDB is an open-source, in-memory database system optimized for fast analytical querying and low memory usage. It's lightweight, embeddable, and seamlessly integrates with popular programming languages like Python and R. DuckDB excels in executing rapid SQL queries on large datasets, making it ideal for data exploration and interactive analysis tasks.**

In [ ]:
!pip install duckdb

**RDKit is a widely-used open-source toolkit for cheminformatics. It offers various functions for working with chemical structures and data, including molecular representation, substructure searching, and compound library handling. RDKit is popular in pharmaceutical research and drug discovery due to its comprehensive features and easy integration with Python.**

In [ ]:
!pip install rdkit

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from tqdm import tqdm
import duckdb
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import average_precision_score

from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier, VotingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
%%time
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

In [ ]:
df.head()

Further, we convert SMILES representations of molecules into RDKit molecule objects and then generates ECFP fingerprints for each molecule, storing the results in the DataFrame. 

**ECFP, or Extended Connectivity Fingerprints**, are molecular fingerprints used in cheminformatics to encode structural features of molecules. They represent connectivity patterns of atoms by hashing local environments within a specified radius. ECFP fingerprints are widely used for similarity searching and activity prediction in drug discovery and cheminformatics due to their efficiency and robustness.

<center><img src="https://miro.medium.com/max/652/1*YjmaYuWldV2yFH1TvPvNNw.jpeg" ></center><br>

In [ ]:
%%time
# Convert SMILES to RDKit molecules
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

# Generate ECFPs
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)

In [ ]:
df['ecfp'].head()

In [ ]:
RANDOM_STATE = 42

In [ ]:
%%time
# One-hot encode the protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Combine ECFPs and one-hot encoded protein_name
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Split the data into train, validation and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_STATE)

In [ ]:
# # Define base models
# rf_model = RandomForestClassifier()
# xgb_model = XGBClassifier()
# lgbm_model = LGBMClassifier()
# catboost_model = CatBoostClassifier()
# svm_model = SVC(kernel='rbf')  # RBF kernel SVM
# mlp_model = MLPClassifier()  # DNN

# # Define ensemble methods
# estimators = [
#     ('rf', rf_model),
#     ('xgb', xgb_model),
#     ('lgbm', lgbm_model),
#     ('catboost', catboost_model),
#     ('svm', svm_model),
#     ('mlp', mlp_model)
# ]

# # Stacking
# stacking_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

# # Blending
# blending_model = VotingClassifier(estimators=estimators, voting='soft')

# # Bagging
# bagging_model = BaggingClassifier(base_estimator=rf_model, n_estimators=10)

# # Train and evaluate each model
# models = {
#     'Random Forest': rf_model,
#     'XGBoost': xgb_model,
#     'LightGBM': lgbm_model,
#     'CatBoost': catboost_model,
#     #'SVM (RBF Kernel)': svm_model,
#     'DNN': mlp_model,
#     'Stacking': stacking_model,
#     'Blending': blending_model,
#     'Bagging': bagging_model
# }

# for name, model in models.items():
#     model.fit(X_train, y_train)
#     y_pred = model.predict_proba(X_test)[:, 1]
#     accuracy = average_precision_score(y_test, y_pred)
#     print(f'{name} Accuracy: {accuracy}')

In [ ]:
NUMBER_OF_MODELS = 3
# Create and train the ensemble models
catboost_model = CatBoostClassifier(iterations=500, random_state=RANDOM_STATE)
lgbm_model = LGBMClassifier(random_state=RANDOM_STATE)
xgb_model = XGBClassifier(random_state=RANDOM_STATE, n_jobs=-1,)

models = [catboost_model, lgbm_model, xgb_model]
for model in models:
    model.fit(X_train, y_train)

In [ ]:
predictions_test = []
# Calibrate probabilities on validation set
for model in models:
    calibrated_model = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
    calibrated_model.fit(X_val, y_val)
    # Evaluate calibrated model on test set
    calibrated_probabilities = calibrated_model.predict_proba(X_test)[:, 1]
    predictions_test.append(calibrated_probabilities)
    
# Ensemble predictions for the test set
ensemble_predictions_test = np.mean(predictions_test, axis=0)

In [ ]:
# Calculate the mean average precision
map_score = average_precision_score(y_test, ensemble_predictions_test)
print(f"Mean Average Precision (mAP): {map_score:.8f}")

In [ ]:
import os
# Process the test.parquet file chunk by chunk sinc the file is huge
test_file = '/kaggle/input/leash-BELKA/test.csv'
output_file = 'submission.csv'  # Specify the path and filename for the output file



# Read the test.csv file into a pandas DataFrame
# Wrap the loop with tqdm to display progress
for df_test in tqdm(pd.read_csv(test_file, chunksize=10_000)):
    
    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(generate_ecfp)

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test['protein_name'].values.reshape(-1, 1))

    # Combine ECFPs and one-hot encoded protein_name
    X_test = [ecfp + protein for ecfp, protein in zip(df_test['ecfp'].tolist(), protein_onehot.tolist())]
    
    predictions_test = []
    # Calibrate probabilities on validation set
    for model in models:
        calibrated_model = CalibratedClassifierCV(model, method='sigmoid', cv='prefit')
        calibrated_model.fit(X_val, y_val)
        # Evaluate calibrated model on test set
        calibrated_probabilities = calibrated_model.predict_proba(X_test)[:, 1]
        predictions_test.append(calibrated_probabilities)

    # Ensemble predictions for the test set
    ensemble_predictions_test = np.mean(predictions_test, axis=0)

#     # Predict the probabilities using ensemble models
#     probabilities_catboost = catboost_model.predict_proba(X_test)[:, 1]
#     probabilities_lgbm = lgbm_model.predict_proba(X_test)[:, 1]
#     probabilities_xgb = xgb_model.predict_proba(X_test)[:, 1]

#     # Average the predictions from the ensemble
#     probabilities_ensemble = (probabilities_catboost + probabilities_lgbm + probabilities_xgb) / NUMBER_OF_MODELS
    
     # Append the probabilities to the list

# Create a DataFrame with 'id' and 'probability' columns
    output_df = pd.DataFrame({'id': df_test['id'], 'binds': ensemble_predictions_test})

# Save the output DataFrame to a CSV file
    output_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))